# Fixture Count Example

- Get your personal Signal Ocean API subscription key (acquired [here](https://apis.signalocean.com/profile)) and replace it below:

In [ ]:
signal_ocean_api_key = ""  # replace with your subscription key

## Setup

### Import libraries and set default values

- Install the Signal Ocean SDK and import all required modules:

In [ ]:
!pip install signal-ocean

from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
from signal_ocean.voyages import VesselClass, VesselClassFilter
import pandas as pd
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import OrderedDict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import math

# Initialize the voyage api
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

#set up the grouped areas
area_mapping = {
    'Arabian Gulf':'AG',
    'Red Sea':'AG',
    'Singapore / Malaysia':'AG',
    'Canada Atlantic Coast':'Caribs',
    'Caribs':'Caribs',
    'US Atlantic Coast':'Caribs',
    'East Coast Mexico':'USG',
    'US Gulf':'USG',
    'Africa Atlantic Coast':'WAF',
    'Argentina & Uruguay':'WAF',
    'Brazil':'WAF'
}

     |████████████████████████████████| 94 kB 1.4 MB/s 


### Set parameters

- set  the vessel class

In [ ]:
vessel_class_name_like = 'vlcc'

- set the year and month from when you want to count fixtures

In [ ]:
year, month = 2022, 2

## Calculate Fixture Count

* get the voyages data using Voyage API 
* we extract the voyages which have start after 2 months before the fixture date
* from the Voyage Events we extract the Area Level 0, which will be used to create the area groups for fixtures

In [ ]:
def get_voyages(vcn, year, month):
  def get_voyage_load_area(voyage_events):
    return next((e.area_name_level0 for e in voyage_events or [] if e.purpose=='Load'), None)
  def get_voyage_port_id(voyage_events):
    return next((e.port_id for e in voyage_events or [] if e.purpose=='Load'), None)

  #calculate the voyage start date (2 month before)
  months_back = 2
  fixture_date_from = date(year=year, month=month,day=1)
  date_from = fixture_date_from - relativedelta(months=+months_back)

  # get the vessel class id
  vc = api.get_vessel_classes(VesselClassFilter(vcn))[0]
  vessel_class_id = vc.vessel_class_id
  
  data = api.get_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
  voyages = pd.DataFrame([v.__dict__ for v in data])
  voyages['Level0AreaName_load'] = voyages['events'].apply(get_voyage_load_area)
  voyages['port_id'] = voyages['events'].apply(get_voyage_port_id)
  voyages.rename(columns={'vessel_class_id': 'VesselClassID'}, inplace=True)
  return voyages

- Using the area_mapping dictionary we group the level 0 areas
- In order a voyage to be fixed, the laycan_from field should not be null
- We keep the voyages which has laycan_From greater than the selected date 
- The final Step is to group the data by year, month, area, and decate and calculate the fixture count for each group

In [ ]:
def preprocess(data,area_mapping):

  # Keep only the dirty
  voyages = data.copy()
  voyages = voyages[voyages.trade_id == 1]

  # merge with the area_mapping on area_level0_name and vessel_class_id
  voyages['Area'] = voyages.Level0AreaName_load.map(lambda x: area_mapping[x] 
                                                    if x in area_mapping 
                                                    else None)
  voyages_df = voyages[["imo","voyage_number","VesselClassID",
                        "Level0AreaName_load","Area","laycan_from","port_id"]]

  # filter with laycan from is not null
  voyages_df = voyages_df[~voyages_df.laycan_from.isna()]
  
  # filter with laycan from
  filtered_voyages = voyages_df[voyages_df.laycan_from.dt.date >= 
                                date(year=year, month=month, day=1)]

  # split the laycan from to month, date, year and create the decade
  final_df = filtered_voyages.copy()
  final_df['day'] = final_df['laycan_from'].dt.day
  final_df['month'] = final_df['laycan_from'].dt.month
  final_df['year'] = final_df['laycan_from'].dt.year

  choices = [1,2,3]
  conditions = [
        (final_df['day'] <= 10), 
        ((final_df['day'] > 10) & (final_df['day'] <= 20)),
        (final_df['day'] > 20) ]
  final_df['decade'] = np.select(conditions, choices, default=np.nan)
  final_df['decade'] = final_df['decade'].astype(int)


  # Remove 'Other' load areas
  final_df = final_df[~final_df.Area.isna()]

  
  fixture_counts_df = (
    final_df
    .groupby(['month','year','Area','decade'])
    .size()
    .rename('fixture_counts')
    .reset_index()    
  )

  month_groups = set(fixture_counts_df.groupby(['year','month']) \
                                        .groups.keys())
  area_groups = set(area_mapping.values())
  area_month_groups = []
  for area in area_groups:
    for period in month_groups:
      area_month_groups.append((period[0],period[1],area))
  decate_areas = []
  for area_month in area_month_groups:
    decate_areas.append((1,area_month[0],area_month[1],area_month[2]))
    decate_areas.append((2,area_month[0],area_month[1],area_month[2]))
    decate_areas.append((3,area_month[0],area_month[1],area_month[2]))
  decate_per_area = pd.DataFrame(decate_areas, columns=['decade', 
                                                        'year','month',
                                                        'Area'])
  fixture_counts_df = pd.merge(fixture_counts_df,decate_per_area,
                               on=['decade','year','month','Area'],
                               how='right').fillna(0)

  fixture_counts_df['Area Total'] = (
    fixture_counts_df
    .groupby(['year','month','Area'])
    .fixture_counts
    .transform('sum')    
  )
  
  fixture_counts_df = fixture_counts_df.sort_values(
      ['year','month','Area Total','Area','decade'],
      ascending=[False,False,True,True,True,]) 
  return fixture_counts_df

- Plot Results

In [ ]:
def plot_results(fixture_counts_df):

  decade_mapping = {
      1:'First decade (1st - 10th)',
      2:'Second decade (10th - 20th)',
      3:'Third decade (20th - 30th)'   
  }
  fixture_counts_df['year'] = fixture_counts_df['year'].astype(int)
  fixture_counts_df['month'] = fixture_counts_df['month'].astype(int)
  fixture_counts_df['year_month_name'] = \
    fixture_counts_df.apply(lambda x: \
                              date(x.year, x.month, 1).strftime('%B') \
                              + " " + str(x.year), axis=1) 

  fig = px.bar(fixture_counts_df, 
                  x='fixture_counts', 
                  y='Area',
                  color=fixture_counts_df['decade'].map(decade_mapping),  
                  barmode='stack', 
                  orientation='h',
                  title='Fixture counts',
                  text='fixture_counts',
                  facet_col  ="year_month_name",
                  facet_col_wrap = 2,
                  )

  rows = len(fixture_counts_df.groupby(['year','month']).groups.keys())
  fig.update_layout(width=1400,
                    height=250*math.ceil(rows/2),
                    legend_title="Decade",
                    xaxis={'categoryorder':'total ascending'})
  fig.show()

## Results

In [ ]:
# Get voyages
voyages = get_voyages(vessel_class_name_like, year, month)

In [ ]:
# Preprocess and plot the results
preprocessed_voyages = preprocess(voyages, area_mapping)
preprocessed_voyages.head()

,month,year,Area,decade,fixture_counts,Area Total
9,5,2022,AG,1,0.0,0.0
10,5,2022,AG,2,0.0,0.0
11,5,2022,AG,3,0.0,0.0
21,5,2022,Caribs,1,0.0,0.0
22,5,2022,Caribs,2,0.0,0.0


In [ ]:
plot_results(preprocessed_voyages)